In [1]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import (
    Audio, display, clear_output)
from ipywidgets import widgets
from functools import partial
%matplotlib inline
import random
from scipy.signal import square

In [5]:
rate = 16000.
duration = 0.5
t = np.linspace(
    0., duration, int(rate * duration))

notes = 'C,C#,D,D#,E,F,F#,G,G#,A,A#,B,C2'.split(',')
# notes = 'C,D,D#,F,G,A,A#,C2'.split(',')
keylist = np.array(notes)

freqs = 440. * 2**(np.arange(3, 3 + len(notes)) / 12.)
notes = list(zip(notes, freqs))
notes_d = {}
for i in range(len(notes)):
    notes_d[notes[i][0]] = notes[i][1]
notes = notes_d

In [15]:
def synth(f):
    x = np.sin(f * 2. * np.pi * t)
    display(Audio(x, rate=rate, autoplay=True))
    
def synth2(f1, f2):
    x = np.sin(f1 * 2. * np.pi * t) + np.sin(f2 * 2. * np.pi * t)
    display(Audio(x, rate=rate, autoplay=True))
    
def synth_d(note):
    f = notes[note]
#     x = np.sin(f * 2. * np.pi * t)
    x = square(f * 2. * np.pi * t)
    display(Audio(x, rate=rate, autoplay=True))

def piano_d(note):
    f = notes[note]
    x = np.exp(-10*t)*(-0.25*np.sin(f * 3. * np.pi * t) + 0.25*np.sin(f * np.pi * t) + np.sqrt(3)/2.*np.sin(f * np.pi * t))
    display(Audio(x, rate=rate, autoplay=True))
    
def play_chord(notes_, duration, mode='harmonic'):
    t = np.linspace(0., duration, int(rate * duration))
    
    if mode.lower() in ['harmonic']:
        x = 0
        for note in notes_:
            f = notes[note]
            x += np.exp(-10*t)*(-0.25*np.sin(f * 3. * np.pi * t) + 0.25*np.sin(f * np.pi * t) + np.sqrt(3)/2.*np.sin(f * np.pi * t))
    if mode.lower() in ['melodic']:
        x = np.array([])
        for note in notes_:
            f = notes[note]
            x = np.append(x, np.exp(-10*t)*(-0.25*np.sin(f * 3. * np.pi * t) + 0.25*np.sin(f * np.pi * t) + np.sqrt(3)/2.*np.sin(f * np.pi * t)))
    
    display(Audio(x, rate=rate, autoplay=True))
    
def random_chord(n=3):
    keylist_r = np.random.permutation(keylist)
    chord = keylist_r[:n]
    return chord

def random_interval():
    r = np.random.randint(len(keylist)) + 1
    chord = ['C']
    chord.append(keylist[r])
    return chord, r

def random_melody(n):
    notes = []
    nr = -1
    for i in range(n):
        r = np.random.randint(len(keylist)-1) + 1
        if nr == -1:
            nr = r
        while np.abs(nr-r) < 1:
            r = np.random.randint(len(keylist)-1) + 1
        nr = r
        notes.append(keylist[r])
    return notes

In [14]:
rand_notes = random_melody(100)
play_chord(rand_notes, 0.1, mode='melodic')

In [5]:
# r_chord = random_chord(2)
interv, _ = random_interval()
print(interv)
play_chord(interv, 1, mode='harmonic')
# synth_d('C')

['C', 'D#']


In [3]:
intervals = ['2m', '2M', '3m', '3M', '4', '4+', '5', '6m', '6M', '7m', '7M', '8']
def interval_guess(mode="harmonic"):
    interval, r = random_interval()
    play_chord(interval, 1, mode=mode)
    guess = input('Interval: ')
    true = intervals[r-1]
    if guess in [true]:
        print('YES')
        res = 1
    else:
        print('NO, IT WAS', true)
        res = 0

    return res

def several_interval_guess(n, mode="harmonic"):
    score = 0
    for i in range(n):
        res = interval_guess()
        score += res
    
    return score

In [4]:
# print(several_interval_guess(20, mode="harmonic"), "/20")

In [4]:
import pygame.midi

def print_devices():
    for n in range(pygame.midi.get_count()):
        print (n,pygame.midi.get_device_info(n))

def number_to_note(number):
    notes = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
    return notes[number%12]

def readInput(input_device):
    while True:
        if input_device.poll():
            event = input_device.read(1)[0]
            data = event[0]
            timestamp = event[1]
            note_number = data[1]
            velocity = data[2]
            if velocity != 0:
                print (number_to_note(note_number), velocity)
                play_chord([number_to_note(note_number)], 0.2, mode='melodic')
            
            

pygame.midi.init()
my_input = pygame.midi.Input(1) #only in my case the id is 2
readInput(my_input)